In [1]:
import time
import torch
import numpy as np
import torch.nn as nn
from torch import optim
from utils import fidelity
from QudiTop.gates import *
from numpy.linalg import norm
from QudiTop.circuit import Circuit
from QudiTop.global_var import DTYPE
from scipy.stats import unitary_group
from QudiTop.expectation import Expectation

np.set_printoptions(linewidth=200)
torch.set_printoptions(linewidth=200)

In [73]:
d = 3
circ = Circuit(d, 1) + RZ(d, [0, 1], 1).on(0) + RY(d, [0, 1], 2).on(0) + RZ(d, [0, 1], 3).on(0)
print(circ.matrix())

[[-0.22484513-0.49129552j -0.45464873+0.70807344j  0.        +0.j        ]
 [ 0.45464873+0.70807344j -0.22484513+0.49129552j  0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j        ]]


In [67]:
d, m = 3, 2
circ = Circuit(d, 2) + RZ(d, [0, 1], 1).on(0, 1, m) + RY(d, [0, 2], 2).on(0, 1, m) + RZ(d, [0, 1], 3).on(0, 1, m)
print(circ.matrix().real)
print(circ.matrix().imag)

[[ 1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.         -0.22484513  0.         -0.0595233 ]
 [ 0.          0.          0.          0.          0.          0.          0.         -0.41614684  0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.73846024  0.          0.54030234]]


In [59]:
d, m = 3, 0
circ = Circuit(d, 2)

a = 1
ind = [0, 1]
circ = circ + RZ(d, ind, a).on(0, 1, m)
print(RZ(d, ind, a).matrix())
# print(circ.matrix())

b = 2
ind = [0, 2]
circ = circ + RZ(d, ind, b).on(0, 1, m)
print(RZ(d, ind, b).matrix())
# print(circ.matrix())

circ += GP(d, 3).on(0, 1, m)
print(circ.matrix())

tensor([[0.8776-0.4794j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.8776+0.4794j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 1.0000+0.0000j]])
tensor([[0.5403-0.8415j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 1.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.5403+0.8415j]])
[[-0.21079582+0.97753018j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.80114365-0.59847218j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.41614687-0.90929741j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.      

In [58]:
d, m = 3, 0
circ = Circuit(d, 2)

a = 1
ind = [0, 1]
circ = circ + RZ(d, ind, a / 2).on(0) + X(d, ind).on(0, 1, m) + RZ(d, ind, -a / 2).on(0) + X(d, ind).on(0, 1, m)
print(RZ(d, ind, a).matrix())
# print(circ.matrix())

b = 2
ind = [0, 2]
circ = circ + RZ(d, ind, b / 2).on(0) + X(d, ind).on(0, 1, m) + RZ(d, ind, -b / 2).on(0) + X(d, ind).on(0, 1, m)
print(RZ(d, ind, b).matrix())
# print(circ.matrix())

circ += GP(d, 3).on(0, 1, m)
print(circ.matrix())

tensor([[0.8776-0.4794j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.8776+0.4794j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 1.0000+0.0000j]])
tensor([[0.5403-0.8415j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 1.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.5403+0.8415j]])
[[-0.21079573+0.97753024j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.80114365-0.59847218j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.41614679-0.90929747j  0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.      

In [19]:
d, nq = 3, 2
circ = Circuit(d, nq)
for i in range(d):
    circ += RY(d, [1, 2], np.pi / (2 + i)).on(1, 0, d - i - 1)
print(circ)
print(circ.matrix())

Circuit(
  (gates): ModuleList(
    (0): RY(3 [1 2] π/2|1 <-: 0 - 2)
    (1): RY(3 [1 2] π/3|1 <-: 0 - 1)
    (2): RY(3 [1 2] π/4|1 <-: 0 - 0)
  )
)
[[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.9238795 +0.j  0.        +0.j  0.        +0.j -0.38268346+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.86602539+0.j  0.        +0.j  0.        +0.j -0.5       +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j  0.70710677+0.j  0.        +0.j  0

In [3]:
def Cd(d, pr, state, obj, ctrl):
    if d != 3:
        raise ValueError('Only works when d = 3')
    if state < 0 or state >= d:
        raise ValueError(f'¦{state}⟩ control state should in 0 to {d-1}')
    circ = Circuit(d, nq)
    circ += RZ(d, [0, 1], f'{pr}RZ01').on(obj)
    circ += X(d, [0, 1]).on(obj, ctrl, state)
    circ += RZ(d, [0, 1], f'{pr}-RZ01').on(obj)
    circ += X(d, [0, 1]).on(obj, ctrl, state)
    circ += RZ(d, [0, 2], f'{pr}RZ02').on(obj)
    circ += X(d, [0, 2]).on(obj, ctrl, state)
    circ += RZ(d, [0, 2], f'{pr}-RZ02').on(obj)
    circ += X(d, [0, 2]).on(obj, ctrl, state)
    circ += GP(d, f'{pr}phase_obj').on(obj)
    circ += GP(d, f'{pr}phase_ctrl').on(ctrl)
    return circ


def qutrit_ansatz(gate: UMG, with_phase: bool = False):
    obj = gate.obj_qudits
    name = f'{gate.name}_'
    circ = Circuit(d, nq)
    index = [[0, 2], [1, 2], [0, 2]]
    if len(obj) == 1:
        for i, ind in enumerate(index):
            str_pr = f'{"".join(str(i) for i in ind)}_{i}'
            circ += RZ(d, ind, f'{name}RZ{str_pr}').on(obj[0])
            circ += RY(d, ind, f'{name}RY{str_pr}').on(obj[0])
            circ += RZ(d, ind, f'{name}Rz{str_pr}').on(obj[0])
    elif len(obj) == 2:
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U1').on(obj[1]))
        circ += Cd(d, f'{name}Cd1', 1, obj[1], obj[0])
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U2').on(obj[1]))
        circ += Cd(d, f'{name}Cd2', 2, obj[1], obj[0])
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U3').on(obj[1]))
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[0], obj[1], 0)
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U4').on(obj[1]))
        circ += Cd(d, f'{name}Cd3', 2, obj[1], obj[0])
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U5').on(obj[1]))
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[0], obj[1], 1)
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U6').on(obj[1]))
        circ += Cd(d, f'{name}Cd4', 0, obj[1], obj[0])
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U7').on(obj[1]))
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[0], obj[1], 2)
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U8').on(obj[1]))
        circ += Cd(d, f'{name}Cd5', 2, obj[1], obj[0])
        circ += qutrit_ansatz(UMG(d, np.eye(d), name=f'{name}U9').on(obj[1]))
    else:
        raise ValueError('Only works when nq <= 2')
    if with_phase:
        circ += [GP(d, 'phase').on(i) for i in obj]
    return circ


d, nq = 3, 1
circ = Circuit(d, nq)
ansatz = Circuit(d, nq)
mat = unitary_group.rvs(d**nq, random_state=42)
obj = list(range(nq))
gate = UMG(d, mat, name=f'mat').on(obj)
circ += gate
ansatz += qutrit_ansatz(gate)
print(ansatz)

pr = ansatz.get_parameters()
g_num = len(ansatz.gates)
p_num = len(pr)
print('Number of qudits: %d' % nq)
print('Number of params: %d' % p_num)
print('Number of gates: %d' % g_num)

psi = circ.get_qs()
rho = np.outer(psi, psi.conj())
print('Hamiltonian Dimension:', rho.shape)
Ham = [(1, UMG(d, rho).on(obj[::-1]))]
expect = Expectation(Ham)

Circuit(
  (gates): ModuleList(
    (0): RZ(3 [0 2] mat_RZ02_0|0)
    (1): RY(3 [0 2] mat_RY02_0|0)
    (2): RZ(3 [0 2] mat_Rz02_0|0)
    (3): RZ(3 [1 2] mat_RZ12_1|0)
    (4): RY(3 [1 2] mat_RY12_1|0)
    (5): RZ(3 [1 2] mat_Rz12_1|0)
    (6): RZ(3 [0 2] mat_RZ02_2|0)
    (7): RY(3 [0 2] mat_RY02_2|0)
    (8): RZ(3 [0 2] mat_Rz02_2|0)
  )
)
Number of qudits: 1
Number of params: 9
Number of gates: 9
Hamiltonian Dimension: (3, 3)


In [4]:
start = time.perf_counter()
p0 = np.random.uniform(-1, 1, p_num)
target = torch.tensor([1], dtype=DTYPE)
ansatz.assign_ansatz_parameters(dict(zip(pr, p0)))
optimizer = optim.Adam(ansatz.parameters(), lr=1e-1)
for i in range(1000):
    out = expect(ansatz())
    loss = nn.L1Loss()(out, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        t = time.perf_counter() - start
        print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))
    if loss < 1e-8:
        break
t = time.perf_counter() - start
print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))

pr_res = ansatz.get_parameters()
psi_res = ansatz.get_qs(pr_res)
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res)**2)

Loss: 0.917853832244873, Fidelity: 0.082146182656288,   0, 0.8764
Loss: 0.536430597305298, Fidelity: 0.463569402694702,  10, 0.9968
Loss: 0.459062159061432, Fidelity: 0.540937840938568,  20, 1.1112
Loss: 0.248667240142822, Fidelity: 0.751332759857178,  30, 1.2401
Loss: 0.005841374397278, Fidelity: 0.994158625602722,  40, 1.3542
Loss: 0.029161632061005, Fidelity: 0.970838367938995,  50, 1.4781
Loss: 0.003436625003815, Fidelity: 0.996563374996185,  60, 1.5935
Loss: 0.001842975616455, Fidelity: 0.998157024383545,  70, 1.7143
Loss: 0.001288652420044, Fidelity: 0.998711347579956,  80, 1.8292
Loss: 0.000018060207367, Fidelity: 0.999981939792633,  90, 1.9383
Loss: 0.000105857849121, Fidelity: 0.999894142150879, 100, 2.0492
Loss: 0.000063002109528, Fidelity: 0.999936997890472, 110, 2.1647
Loss: 0.000005006790161, Fidelity: 0.999994993209839, 120, 2.2924
Loss: 0.000002026557922, Fidelity: 0.999997973442078, 130, 2.4095
Loss: 0.000002622604370, Fidelity: 0.999997377395630, 140, 2.5286
Loss: 0.00